In [1]:
# Ячейка 1: ПРИНУДИТЕЛЬНОЕ ОБНОВЛЕНИЕ БИБЛИОТЕК
!apt-get update -qq
!apt-get install -y -qq djvulibre-bin
# Флаг -U обязателен, чтобы получить доступ к gemini-1.5-flash
!pip install -U -q google-generativeai telethon pymupdf tqdm pillow
!echo "✅ Библиотеки ОБНОВЛЕНЫ. Теперь модель должна быть видна."

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package djvulibre-bin.
(Reading database ... 128639 files and directories currently installed.)
Preparing to unpack .../djvulibre-bin_3.5.28-2ubuntu0.22.04.1_amd64.deb ...
Unpacking djvulibre-bin (3.5.28-2ubuntu0.22.04.1) ...
Selecting previously unselected package libexiv2-27:amd64.
Preparing to unpack .../libexiv2-27_0.27.5-3ubuntu1_amd64.deb ...
Unpacking libexiv2-27:amd64 (0.27.5-3ubuntu1) ...
Selecting previously unselected package libgraphicsmagick-q16-3.
Preparing to unpack .../libgraphicsmagick-q16-3_1.4+really1.3.38-1ubuntu0.1_amd64.deb ...
Unpacking libgraphicsmagick-q16-3 (1.4+really1.3.38-1ubuntu0.1) ...
Selecting previously unselected package libgraphicsmagick++-q16-12.
Preparing to unpack .../libgraphicsmagick++-q16-12_1.4+really1.3.38-1ubuntu0.1_amd64.de

In [2]:
# Ячейка Диагностики
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

print("🔍 Сканирование доступных моделей...")
try:
    user_secrets = UserSecretsClient()
    GENAI_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=GENAI_KEY)
    
    available_models = []
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f"   ✅ Доступна: {m.name}")
            available_models.append(m.name)
            
    if not available_models:
        print("❌ Нет доступных моделей! Проверьте API Key.")
    else:
        print(f"\nВсего найдено: {len(available_models)}")

except Exception as e:
    print(f"❌ Критическая ошибка: {e}")
    print("💡 СОВЕТ: Нажмите в меню 'Run' -> 'Restart Session', чтобы обновить библиотеки.")

🔍 Сканирование доступных моделей...
   ✅ Доступна: models/gemini-2.5-pro-vtea-da-csi
   ✅ Доступна: models/gemini-2.5-pro-preview-03-25
   ✅ Доступна: models/gemini-2.5-flash
   ✅ Доступна: models/gemini-2.5-pro-preview-05-06
   ✅ Доступна: models/gemini-2.5-pro-preview-06-05
   ✅ Доступна: models/gemini-2.5-pro
   ✅ Доступна: models/gemini-2.0-flash-exp
   ✅ Доступна: models/gemini-2.0-flash
   ✅ Доступна: models/gemini-2.0-flash-001
   ✅ Доступна: models/gemini-2.0-flash-exp-image-generation
   ✅ Доступна: models/gemini-2.0-flash-lite-001
   ✅ Доступна: models/gemini-2.0-flash-lite
   ✅ Доступна: models/gemini-2.0-flash-lite-preview-02-05
   ✅ Доступна: models/gemini-2.0-flash-lite-preview
   ✅ Доступна: models/gemini-2.0-pro-exp
   ✅ Доступна: models/gemini-2.0-pro-exp-02-05
   ✅ Доступна: models/gemini-exp-1206
   ✅ Доступна: models/gemini-2.0-flash-thinking-exp-01-21
   ✅ Доступна: models/gemini-2.0-flash-thinking-exp
   ✅ Доступна: models/gemini-2.0-flash-thinking-exp-1219
   ✅ Д

In [4]:
# === ЯЧЕЙКА 2: V46 (SMART COVER GUARD & STRICT SAFETY) ===
import os
import time
import json
import asyncio
import shutil
import re
import fitz  # PyMuPDF
import subprocess
import google.generativeai as genai
from PIL import Image
from datetime import datetime
from telethon import TelegramClient, events, functions, types
from telethon.sessions import MemorySession
from kaggle_secrets import UserSecretsClient
from tqdm.notebook import tqdm
from google.api_core.exceptions import ResourceExhausted, ServiceUnavailable, InternalServerError

# === SAFETY ===
Image.MAX_IMAGE_PIXELS = None 

# === SETTINGS ===
SOURCE_USERNAME = "sendDoc39"
BATCH_STRATEGY = [5, 3, 1]  

# GLOBAL HARD LIMIT: Minimum seconds between ANY API calls
GLOBAL_REQUEST_DELAY = 30.0  

ZOOM_FACTOR = 2.0          
MARKER_EMOJI = "✅"
MAX_EMPTY_GAP = 5
SAFETY_LIMIT = 500

# RENDER SETTINGS
DJVU_TARGET_SIZE = 2560    
CROP_PADDING = 15          
SNAP_THRESHOLD = 30        

# --- PRODUCTION SETTINGS ---
DEBUG_PAGE_LIMIT = 3     
# ---------------------------

user_secrets = UserSecretsClient()
TEMP_DIR = "./temp_processing"
CROPS_DIR = "./temp_crops"

LOG_BUFFER = []

def log(text):
    tqdm.write(text)
    LOG_BUFFER.append(text)

async def send_report(client, dest_entity):
    report_text = "\n".join(LOG_BUFFER)
    if len(report_text) > 4000: report_text = report_text[-3900:]
    try:
        if dest_entity: await client.send_message(dest_entity, f"📝 **Report:**\n\n...{report_text}")
    except: pass

# === MODEL MANAGER (HARD THROTTLE) ===
class ModelRotator:
    def __init__(self, api_keys):
        if isinstance(api_keys, str): self.api_keys = [api_keys]
        else: self.api_keys = api_keys
        self.current_key_idx = 0
        self._configure_current_key()
        self.last_request_ts = 0
        
        self.models_list = [
            "models/gemini-2.5-flash",      # Priority 1
            "models/gemini-2.0-flash",      # Priority 2
        ]
        log(f"🧠 Models initialized. Delay set to {GLOBAL_REQUEST_DELAY}s.")

    def _configure_current_key(self):
        genai.configure(api_key=self.api_keys[self.current_key_idx])

    async def _enforce_global_pace(self):
        now = time.time()
        elapsed = now - self.last_request_ts
        if elapsed < GLOBAL_REQUEST_DELAY:
            wait_time = GLOBAL_REQUEST_DELAY - elapsed + 1
            await asyncio.sleep(wait_time)
        self.last_request_ts = time.time()

    async def generate_content(self, content):
        for i, model_name in enumerate(self.models_list):
            await self._enforce_global_pace()
            
            model = genai.GenerativeModel(
                model_name=model_name,
                safety_settings=[{"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}],
                generation_config={"response_mime_type": "application/json", "temperature": 0.1}
            )
            
            try:
                log(f"      📡 [{model_name}] Sending...")
                t_start = time.time()
                response = await asyncio.to_thread(model.generate_content, content)
                dur = time.time() - t_start
                
                usage = {"in": 0, "out": 0}
                if response.usage_metadata:
                    usage["in"] = response.usage_metadata.prompt_token_count
                    usage["out"] = response.usage_metadata.candidates_token_count
                
                return response.text, usage, model_name, dur
            
            except (ResourceExhausted, ServiceUnavailable, InternalServerError) as e:
                log(f"      ⚠️ Limit/Error on {model_name}.")
                if i == len(self.models_list) - 1:
                    log("      ❌ ALL MODELS EXHAUSTED. Aborting execution.")
                    raise e 
                log("      ➡️ Switching to backup model...")
                continue
                
            except Exception as e:
                if "429" in str(e):
                    log(f"      ⚠️ 429 Too Many Requests on {model_name}.")
                    if i == len(self.models_list) - 1:
                        log("      ❌ ALL MODELS EXHAUSTED (429). Aborting.")
                        raise e
                    continue
                log(f"      ❌ Unexpected Error ({model_name}): {e}")
                raise e

        raise Exception("Unexpected logic flow in ModelRotator")

# === INIT ===
model_rotator = None
try:
    GENAI_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    model_rotator = ModelRotator(GENAI_KEY)
except Exception as e: log(f"❌ Init Error: {e}")

# === AI LOGIC ===
def get_gemini_prompt(is_first_batches, previous_context=None):
    base_prompt = """
    ROLE: Expert OCR & Document Analyst.
    TASK: Convert book pages into Structured JSON.
    """
    
    if previous_context:
        base_prompt += f"""
    CONTEXT FROM PREVIOUS BATCH:
    "...{previous_context}..."
    INSTRUCTION: The text above is the end of the previous page. Use it to check for broken sentences.
    """

    base_prompt += """
    OUTPUT FORMAT:
    {
      "pages": [
        {
           "page_number": int,
           "main_text": "string",       // MARKDOWN.
           "marginalia": "string",      // Technical headers/footers
           "footnotes": "string",       // Footnotes
           "media_objects": [
              {
                 "id": "string",        // For Cover use "book_cover"
                 "coordinates": [ymin, xmin, ymax, xmax], 
                 "type": "string",      // "image", "table", "cover"
                 "caption_ru": "string" 
              }
           ]
        }
      ],
    """
    
    if is_first_batches:
        base_prompt += """
      "bibliographic_data": {
          "title": "string",
          "authors": ["string"],        
          "publisher": "string",
          "year": "string",
          "isbn": "string",
          "copyright_holder": "string"
      }
    }
    """
    else:
        base_prompt += '  "batch_metadata": { "info": "continuation" }\n}'

    base_prompt += """
    RULES:
    1. **COVER PAGE (CRITICAL)**: 
       - If Page 1 is the Cover, return EXACTLY ONE media object with type="cover" and id="book_cover".
       - Set coordinates to [0,0,1000,1000].
       - DO NOT split the cover into title/logo/author images. Keep it whole.

    2. **SMART HEADERS**: 
       - Mark headers with `##` or `###`.
       - Look for bold, centered, or separate lines introducing topics.

    3. **TEXT CLEANING**: 
       - **De-hyphenate:** "con-\ntinue" -> "continue".
       - **Reflow:** Remove artificial newlines.
       - **Columns:** Read logical flow (Col 1 -> Col 2).

    4. **INLINE NOTES**:
       - Keep notes inside `main_text` as `> Note: ...`. Do NOT move to footnotes.

    5. **COORDINATES**:
       - Capture FULL image content (0-1000 scale).
    """
    return base_prompt

async def process_single_batch(image_paths, page_nums, prev_ctx_text):
    if not model_rotator: raise Exception("No models")
    
    is_start_of_book = (page_nums[0] <= 10)
    prompt = get_gemini_prompt(is_start_of_book, prev_ctx_text)
    content = [prompt]
    loaded_imgs = []
    
    for p in image_paths:
        img = Image.open(p)
        w, h = img.size
        if w > 3072 or h > 3072:
            img.thumbnail((3072, 3072), Image.Resampling.LANCZOS)
        loaded_imgs.append(img)
        content.append(img)
        
    text_resp, usage, model_name, duration = await model_rotator.generate_content(content)
    
    clean_text = text_resp.replace("```json", "").replace("```", "").strip()
    if clean_text.startswith("json"): clean_text = clean_text[4:].strip()
    match = re.search(r'\{.*\}', clean_text, re.DOTALL)
    if match: clean_text = match.group(0)

    data = json.loads(clean_text) 
    
    batch_meta = data.get("bibliographic_data", data.get("batch_metadata", None))
    return data.get("pages", []), loaded_imgs, usage, model_name, duration, batch_meta

def crop_images(pil_imgs, pages_data):
    crops = []
    if not os.path.exists(CROPS_DIR): os.makedirs(CROPS_DIR, exist_ok=True)
    
    for i, page_data in enumerate(pages_data):
        if i >= len(pil_imgs): break
        media_list = page_data.get("media_objects", [])
        src = pil_imgs[i]
        w, h = src.size
        
        # === COVER GUARD LOGIC ===
        # Check if ANY object on this page is marked as cover
        is_cover_page = False
        cover_object = None
        
        for m in media_list:
            if m.get("type") == "cover" or m.get("id") == "book_cover":
                is_cover_page = True
                cover_object = m
                break
        
        if is_cover_page and cover_object:
            # If cover detected, IGNORE all other objects. 
            # Force Full Page Crop.
            cover_object["coordinates"] = [0, 0, 1000, 1000]
            cover_object["id"] = "book_cover"
            media_list = [cover_object]
        # =========================
        
        for meta in media_list:
            try:
                ymin, xmin, ymax, xmax = meta["coordinates"]
                
                left = (xmin/1000)*w - CROP_PADDING
                top = (ymin/1000)*h - CROP_PADDING
                right = (xmax/1000)*w + CROP_PADDING
                bottom = (ymax/1000)*h + CROP_PADDING
                
                if left < SNAP_THRESHOLD: left = 0
                if top < SNAP_THRESHOLD: top = 0
                if right > (w - SNAP_THRESHOLD): right = w
                if bottom > (h - SNAP_THRESHOLD): bottom = h
                
                left = max(0, left); top = max(0, top)
                right = min(w, right); bottom = min(h, bottom)
                
                if (right-left < 30) or (bottom-top < 30): continue
                
                crop = src.crop((left, top, right, bottom))
                
                raw_id = meta.get("id", f"media_{int(time.time())}_{i}")
                # Sanitize ID
                clean_id = re.sub(r'[^a-zA-Z0-9_]', '', raw_id)
                
                c_name = f"{clean_id}.png"
                c_path = os.path.join(CROPS_DIR, c_name)
                crop.save(c_path)
                
                caption = f"{meta.get('type', 'img').upper()}: {meta.get('caption_ru', '')}"
                crops.append((c_path, caption, clean_id))
            except: pass
    return crops

# === TELEGRAM & UTILS ===
async def get_last_id(client, entity):
    try:
        full = await client(functions.channels.GetFullChannelRequest(entity))
        match = re.search(r"\[LastID:\s*(\d+)\]", full.full_chat.about or "")
        if match: return int(match.group(1))
    except: pass
    return None

async def update_last_id(client, entity, new_id):
    try:
        full = await client(functions.channels.GetFullChannelRequest(entity))
        about = full.full_chat.about or ""
        tag = f"[LastID: {new_id}]"
        if "[LastID:" in about: new_about = re.sub(r"\[LastID:\s*\d+\]", tag, about)
        else: new_about = f"{about}\n\n{tag}".strip()
        if new_about != about: await client(functions.messages.EditChatAboutRequest(peer=entity, about=new_about))
    except: pass

async def is_already_processed(message):
    try:
        if not message.reactions: return False
        for r in message.reactions.results:
            emoji = r.reaction.emoticon if hasattr(r.reaction, 'emoticon') else str(r.reaction)
            if emoji == MARKER_EMOJI and r.chosen: return True
    except: pass
    return False

async def mark_processed(client, msg):
    try:
        await client(functions.messages.SendReactionRequest(peer=msg.peer_id, msg_id=msg.id, reaction=[types.ReactionEmoji(emoticon=MARKER_EMOJI)]))
    except: pass

def render_pdf(path):
    doc = fitz.open(path)
    mat = fitz.Matrix(ZOOM_FACTOR, ZOOM_FACTOR)
    limit = len(doc)
    if DEBUG_PAGE_LIMIT:
        limit = min(len(doc), DEBUG_PAGE_LIMIT)
        log(f"   🛠 DEBUG: Processing first {limit} pages only.")
    for i in range(limit):
        page = doc.load_page(i)
        pix = page.get_pixmap(matrix=mat, alpha=False)
        out = os.path.join(TEMP_DIR, f"page_{i+1:04d}.png")
        pix.save(out)
        yield out, i+1, len(doc)
    doc.close()

def render_djvu(path):
    safe = os.path.join(TEMP_DIR, "temp.djvu")
    shutil.move(path, safe)
    
    log("   ⏳ Rasterizing DjVu (subprocess)...")
    try:
        subprocess.run([
            'ddjvu', '-format=tiff', f'-size={DJVU_TARGET_SIZE}x{DJVU_TARGET_SIZE}', 
            '-eachpage', safe, os.path.join(TEMP_DIR, "p_%04d.tif")
        ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE)
    except Exception as e:
        log(f"❌ DjVu Render Error: {e}")
        return

    files = sorted([f for f in os.listdir(TEMP_DIR) if f.endswith(".tif")])
    if DEBUG_PAGE_LIMIT: files = files[:DEBUG_PAGE_LIMIT]
    
    log(f"   🖼 Converting {len(files)} pages to PNG...")
    for i, f in enumerate(tqdm(files, desc="Conv", unit="pg", leave=False)):
        full = os.path.join(TEMP_DIR, f)
        try:
            with Image.open(full) as img:
                png = full.replace(".tif", ".png")
                img.save(png, "PNG")
                yield png, i+1, len(files)
            os.remove(full)
        except: pass
    if os.path.exists(safe): os.remove(safe)

# === MAIN ===
async def main_logic():
    log(f"🚀 **Launch V46 (Smart Cover Guard):** {datetime.now().strftime('%H:%M:%S')}")
    
    try:
        API_ID = int(user_secrets.get_secret("TELEGRAM_API_ID"))
        API_HASH = user_secrets.get_secret("TELEGRAM_API_HASH")
        BOT_TOKEN = user_secrets.get_secret("TELEGRAM_BOT_TOKEN")
        DEST_ID = int(user_secrets.get_secret("DEST_CHANNEL_ID"))
    except: return log("❌ Secrets missing.")

    for d in [TEMP_DIR, CROPS_DIR]:
        if os.path.exists(d): shutil.rmtree(d); os.makedirs(d, exist_ok=True)

    client = TelegramClient(MemorySession(), API_ID, API_HASH)
    await client.start(bot_token=BOT_TOKEN)
    log("✅ Telegram OK.")
    if not model_rotator: return await client.disconnect()

    try:
        src_ent = await client.get_entity(SOURCE_USERNAME)
        dst_ent = await client.get_entity(DEST_ID)
    except: return await client.disconnect()

    start_id = await get_last_id(client, src_ent) or 1
    curr_id = start_id + 1
    processed = 0
    empty_streak = 0
    t_in, t_out = 0, 0
    
    pbar = tqdm(total=SAFETY_LIMIT, desc="Scan", unit="msg")

    while processed < SAFETY_LIMIT:
        if empty_streak >= MAX_EMPTY_GAP: log("🏁 End."); break
        try:
            msgs = await client.get_messages(src_ent, ids=[curr_id])
            msg = msgs[0] if msgs else None
            
            if not msg or isinstance(msg, types.MessageEmpty):
                empty_streak += 1; curr_id += 1; pbar.update(1); continue
            
            if await is_already_processed(msg):
                await update_last_id(client, src_ent, msg.id)
                curr_id += 1; pbar.update(1); continue

            fname = getattr(msg.file, 'name', None)
            if not fname: curr_id += 1; pbar.update(1); continue
            ext = fname.split('.')[-1].lower()
            if ext not in ['pdf', 'djvu']: curr_id += 1; pbar.update(1); continue

            empty_streak = 0
            log(f"\n📘 **{fname}** (ID: {msg.id})")
            fpath = os.path.join(TEMP_DIR, fname)
            await msg.download_media(file=fpath)
            
            iter_pages = render_pdf(fpath) if ext == 'pdf' else render_djvu(fpath)
            
            # --- GLOBAL ACCUMULATORS ---
            global_text_blocks = []     
            global_resources = []       
            global_footnotes = []
            global_marginalia = []
            full_bibliographic_data = {}
            
            last_batch_tail_text = None 
            
            # --- ADAPTIVE LOOP ---
            all_page_paths = []
            all_page_idxs = []
            
            current_batch_size = BATCH_STRATEGY[0]
            
            generated_pages_buffer = []
            for p in iter_pages: generated_pages_buffer.append(p)
            
            cursor = 0
            total_pages = len(generated_pages_buffer)
            
            book_failed = False

            while cursor < total_pages:
                end = min(cursor + current_batch_size, total_pages)
                current_slice = generated_pages_buffer[cursor : end]
                slice_paths = [x[0] for x in current_slice]
                slice_idxs = [x[1] for x in current_slice]
                
                log(f"   🔄 Batch {slice_idxs[0]}-{slice_idxs[-1]} (Size: {len(slice_paths)})...")
                
                try:
                    pages_list, imgs, usg, m_name, duration, meta = await process_single_batch(
                        slice_paths, slice_idxs, last_batch_tail_text
                    )
                    
                    log(f"      ✅ [{m_name}] OK in {duration:.1f}s")
                    t_in += usg.get("in",0); t_out += usg.get("out",0)
                    
                    try: await client.send_file(dst_ent, slice_paths, force_document=True)
                    except: pass

                    if meta and isinstance(meta, dict):
                        for k, v in meta.items():
                            if v and not full_bibliographic_data.get(k):
                                full_bibliographic_data[k] = v
                        log(f"      ℹ️ Meta updated. ISBN: {full_bibliographic_data.get('isbn', 'No')}")

                    crops = crop_images(imgs, pages_list)
                    for cp_path, cp_cap, cp_id in crops:
                        msg_link = "local_error"
                        if os.path.exists(cp_path):
                            try:
                                c_msg = await client.send_file(dst_ent, cp_path, caption=f"🖼 {cp_cap}\n🆔 {cp_id}", force_document=False)
                                msg_link = f"tg://msg_id?id={c_msg.id}"
                            except: pass
                            finally:
                                if os.path.exists(cp_path): os.remove(cp_path)
                                await asyncio.sleep(2)
                        global_resources.append({"id": cp_id, "caption": cp_cap, "link": msg_link})

                    for i, page_data in enumerate(pages_list):
                        raw_text = page_data.get("main_text", "")
                        p_num = page_data.get("page_number", slice_idxs[i] if i < len(slice_idxs) else "?")
                        
                        if page_data.get("footnotes"): global_footnotes.append(page_data["footnotes"])
                        if page_data.get("marginalia"): global_marginalia.append(page_data["marginalia"])
                        
                        global_text_blocks.append({"p": p_num, "text": raw_text.strip()})
                    
                    if global_text_blocks:
                        last_text_content = global_text_blocks[-1]["text"]
                        last_batch_tail_text = last_text_content[-1000:]
                    
                    for p in slice_paths:
                        if os.path.exists(p): os.remove(p)
                        
                    cursor += len(slice_paths)
                    
                except Exception as e:
                    log(f"      ⚠️ Failed: {str(e)[:100]}")
                    
                    if "ResourceExhausted" in str(e) or "429" in str(e):
                        log("      ❌ CRITICAL API LIMIT. Stopping book processing.")
                        book_failed = True
                        break

                    new_size = current_batch_size
                    for s in BATCH_STRATEGY:
                        if s < current_batch_size:
                            new_size = s
                            break
                    
                    if new_size == current_batch_size:
                        log(f"      ❌ PARSE ERROR: Page {slice_idxs[0]} failed at size 1. Skipping.")
                        cursor += 1 
                        current_batch_size = 1
                    else:
                        log(f"      📉 Downgrading batch size: {current_batch_size} -> {new_size}")
                        current_batch_size = new_size

            if book_failed:
                await client.send_message(dst_ent, f"❌ **Aborted:** {fname} (API Limits)")
                await update_last_id(client, src_ent, msg.id)
                continue

            # === GLOBAL ASSEMBLY ===
            final_md = f"# {fname}\n\n"
            
            if full_bibliographic_data:
                final_md += "## Metadata\n"
                for k, v in full_bibliographic_data.items():
                    if v: 
                        val_str = ", ".join(v) if isinstance(v, list) else str(v)
                        final_md += f"- **{k}**: {val_str}\n"
                final_md += "\n"
            
            final_md += "## Content\n\n"
            
            joined_content = ""
            for i, block_data in enumerate(global_text_blocks):
                p_num = block_data["p"]
                text = block_data["text"]
                page_marker = f"\n\n> `[Page {p_num}]`\n"
                
                if i > 0:
                    prev_text = global_text_blocks[i-1]["text"]
                    if prev_text and prev_text[-1] not in ".!?\"'”":
                        joined_content += page_marker + text
                    else:
                        joined_content += page_marker + text
                else:
                    joined_content += f"> `[Page {p_num}]`\n" + text
            
            final_md += joined_content.strip()

            if global_resources:
                final_md += "\n\n## APPX: Resources\n"
                for res in global_resources:
                    final_md += f"- **ID**: `{res['id']}`\n"
                    final_md += f"  - **Caption**: {res['caption']}\n"
                    final_md += f"  - **Link**: {res['link']}\n"
            
            if global_footnotes:
                final_md += "\n\n## APPX: Footnotes\n" + "\n\n".join(global_footnotes)
            if global_marginalia:
                final_md += "\n\n## APPX: Marginalia\n" + "\n".join(global_marginalia)

            md_out = os.path.join(TEMP_DIR, f"{os.path.splitext(fname)[0]}.md")
            with open(md_out, "w", encoding="utf-8") as f: f.write(final_md)
            await client.send_file(dst_ent, md_out, caption=f"📚 **MD FULL** (Tok: {t_in+t_out})", force_document=True)
            
            await mark_processed(client, msg)
            await update_last_id(client, src_ent, msg.id)
            log(f"   🎉 Done.")

        except Exception as e:
            log(f"❌ Error ID {curr_id}: {e}")
            import traceback; traceback.print_exc()
        
        finally:
            for d in [TEMP_DIR, CROPS_DIR]:
                if os.path.exists(d): shutil.rmtree(d); os.makedirs(d, exist_ok=True)
        curr_id += 1; processed += 1; pbar.update(1)

    log(f"\n🏁 End. Total Tokens: {t_in}/{t_out}")
    await send_report(client, dst_ent)
    await client.disconnect()

# === RUN ===
print("🟡 Pre-flight check...")
try:
    _ = UserSecretsClient().get_secret("TELEGRAM_API_ID")
    print("🟢 Starting main_logic...")
    await main_logic()
except Exception as e: print(f"🔴 FAIL: {e}")

🧠 Models initialized. Delay set to 30.0s.
🟡 Pre-flight check...
🟢 Starting main_logic...
🚀 **Launch V46 (Smart Cover Guard):** 13:53:08
✅ Telegram OK.


Scan:   0%|          | 0/500 [00:00<?, ?msg/s]


📘 **Грацианский Н. Кёнигсберг (1947), OCR.pdf** (ID: 39)
   🛠 DEBUG: Processing first 3 pages only.
   🔄 Batch 1-3 (Size: 3)...
      📡 [models/gemini-2.5-flash] Sending...
      ✅ [models/gemini-2.5-flash] OK in 200.9s
      ℹ️ Meta updated. ISBN: No
   🎉 Done.

📘 **Маркин_Д_Немцы_Калининграда_2009,_OCR.pdf** (ID: 40)
   🛠 DEBUG: Processing first 3 pages only.
   🔄 Batch 1-3 (Size: 3)...
      📡 [models/gemini-2.5-flash] Sending...
      ✅ [models/gemini-2.5-flash] OK in 64.1s
      ℹ️ Meta updated. ISBN: No
   🎉 Done.
🏁 End.

🏁 End. Total Tokens: 2546/4728
